In [1]:
import numpy as np
import pandas as pd

In [2]:
nytcovid = pd.read_csv("us-states.csv")
colorstate = pd.read_csv("colorstate.csv")
statepop = pd.read_csv("statepop.csv")

In [3]:
statepop["Population"] = statepop["Population"].replace(',','', regex=True).astype(int)
newdf= pd.merge(colorstate, statepop, on='state')

In [4]:
nytcovid['month'] = pd.DatetimeIndex(nytcovid['date']).month
nytcovid['month_name'] = pd.DatetimeIndex(nytcovid['date']).month_name()
nytcovid.head(10)

,date,state,fips,cases,deaths,month,month_name
0,2020-01-21,Washington,53,1,0,1,January
1,2020-01-22,Washington,53,1,0,1,January
2,2020-01-23,Washington,53,1,0,1,January
3,2020-01-24,Illinois,17,1,0,1,January
4,2020-01-24,Washington,53,1,0,1,January
5,2020-01-25,California,6,1,0,1,January
6,2020-01-25,Illinois,17,1,0,1,January
7,2020-01-25,Washington,53,1,0,1,January
8,2020-01-26,Arizona,4,1,0,1,January
9,2020-01-26,California,6,2,0,1,January


In [5]:
covidmonthly = nytcovid[['state','month','cases','month_name','fips']].groupby(['state', 'month'],as_index=False).max()
covidmonthly.head(20)

,state,month,cases,month_name,fips
0,Alabama,3,999,March,1
1,Alabama,4,7068,April,1
2,Alabama,5,17952,May,1
3,Alabama,6,38045,June,1
4,Alabama,7,87723,July,1
5,Alabama,8,126058,August,1
6,Alabama,9,154701,September,1
7,Alabama,10,192285,October,1
8,Alabama,11,247229,November,1
9,Alaska,3,133,March,2


In [6]:
covidmonthly["new_cases"] = covidmonthly[["cases",'state']].groupby(['state']).diff()
covidmonthly.head(20)

,state,month,cases,month_name,fips,new_cases
0,Alabama,3,999,March,1,NaN
1,Alabama,4,7068,April,1,6069.0
2,Alabama,5,17952,May,1,10884.0
3,Alabama,6,38045,June,1,20093.0
4,Alabama,7,87723,July,1,49678.0
5,Alabama,8,126058,August,1,38335.0
6,Alabama,9,154701,September,1,28643.0
7,Alabama,10,192285,October,1,37584.0
8,Alabama,11,247229,November,1,54944.0
9,Alaska,3,133,March,2,NaN


In [7]:
df = pd.merge(covidmonthly, newdf, on='state')

df["case_rate"] = df["new_cases"] / df["Population"] * 100000
df

,state,month,cases,month_name,fips,new_cases,color,Population,case_rate
0,Alabama,3,999,March,1,NaN,r,4903185,NaN
1,Alabama,4,7068,April,1,6069.0,r,4903185,123.776688
2,Alabama,5,17952,May,1,10884.0,r,4903185,221.978163
3,Alabama,6,38045,June,1,20093.0,r,4903185,409.794858
4,Alabama,7,87723,July,1,49678.0,r,4903185,1013.178169
...,...,...,...,...,...,...,...,...,...
477,Wyoming,7,2726,July,56,1239.0,r,578759,214.078744
478,Wyoming,8,3842,August,56,1116.0,r,578759,192.826375
479,Wyoming,9,5948,September,56,2106.0,r,578759,363.882030
480,Wyoming,10,13298,October,56,7350.0,r,578759,1269.958653


In [8]:
hotspots = df[df['case_rate'] > 500].sort_values(by=['month'])
hotspots.head(20)
#df.groupby(['color','month']).sum()
#add up population for new bluestate redstate rate

,state,month,cases,month_name,fips,new_cases,color,Population,case_rate
77,District of Columbia,4,4323,April,11,3828.0,b,705749,542.402469
279,New Jersey,4,118652,April,34,99956.0,b,8882190,1125.353094
297,New York,4,309696,April,36,233485.0,b,19453561,1200.217276
197,Massachusetts,4,62205,April,25,55585.0,b,6892503,806.455942
59,Connecticut,4,27700,April,9,24572.0,b,3565287,689.201178
370,Rhode Island,4,8621,April,44,8133.0,b,1059361,767.726960
78,District of Columbia,5,8801,May,11,4478.0,b,705749,634.503202
198,Massachusetts,5,96965,May,25,34760.0,b,6892503,504.316066
188,Maryland,5,53456,May,24,31631.0,b,6045680,523.200037
371,Rhode Island,5,14928,May,44,6307.0,b,1059361,595.358900


In [9]:
df.to_csv('covidmonthly.csv')
hotspots.to_csv('hotspots.csv')